In [2]:
'''Trains a Bidirectional LSTM on the IMDB sentiment classification task.
Output after 4 epochs on CPU: ~0.8146
Time per epoch on CPU (Core i7): ~150s.
'''

# from __future__ import print_function
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
# from keras.layers.core import Dense, Activation
# from keras.layers.convolutional import Conv1D, MaxPooling1D
# from keras.models import Model
# from keras.layers import Input, Embedding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
max_features = 150
# cut texts after this number of words
# (among top max_features most common words)
maxlen = 100
batch_size = 32

In [4]:
print('Loading data...')
with open('/Users/andradea/Documents/languages/en_US/sentences.pkl', 'rb') as f:
    X, y = pickle.load(f)
    
embedding_matrix = np.recfromtxt('/Users/andradea/Documents/languages/en_US/embeddings.txt',
                                delimiter=',', names=None)

print('Finished loading data!')

Loading data...
Finished loading data!


In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

162519 train sequences
28680 test sequences


In [6]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Pad sequences (samples x time)
x_train shape: (162519, 100)
x_test shape: (28680, 100)


In [9]:
embedding_layer = Embedding(embedding_matrix.shape[0] + 1,
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=maxlen
                            trainable=False)

In [26]:
model = Sequential()
model.add(embedding_layer(128))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[x_test, y_test])

ValueError: Layer embedding_1 was called with an input that isn't a symbolic tensor. Received type: <class 'int'>. Full input: [128]. All inputs to the layer should be tensors.

In [20]:
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

ValueError: Negative dimension size caused by subtracting 5 from 3 for 'conv1d_7/convolution/Conv2D' (op: 'Conv2D') with input shapes: [?,1,3,128], [1,5,128,128].

In [16]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

NameError: name 'model' is not defined